In [1]:
#imports
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk import word_tokenize
import spacy 
import en_core_web_sm

In [3]:
df = pd.read_csv('sentence_classifications.csv')

In [4]:
df.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,sent_#
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,1
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",2
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",3
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",4
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,5


In [6]:
propaganda = df[df['propaganda']=='propaganda']

In [11]:
pd.set_option('max_colwidth', 300)

In [15]:
propaganda[['text','propaganda_type']][1000:1050]

,text,propaganda_type
3926,Everything that the President is DOING belies their argument.,Loaded_Language
3927,"If President Trump were a mole, wouldn’t he have made life easier for Putin?",NaN
3928,Wouldn’t he have eased sanctions?,Doubt
3929,Wouldn’t he have been less confrontational than Obama in Syria?,NaN
3930,Wouldn’t the President be making Putin’s attempts at expanding his influence easier?,NaN
3942,"Instead, President Trump’s POLICIES have actually made life more difficult for Putin at every turn.",Loaded_Language
3949,He actually is pretty tough on Russia.,Repetition
3955,"So if you care about actual actions, I think there’s a lot to be happy about in terms of a tough posture towards Russia.",Repetition
3958,"First, the very people who have been loudest in attacking President Trump about his performance at the Helsinki summit are the people who failed to protect America from Russian meddling in 2016.",Flag-Waving
3959,The very intensity and nastiness of former CIA Director Brennan and former Director of National Intelligence Clapper is an attempt to distract attention from their failure to protect America.,Repetition


In [23]:
prop_type_list = propaganda['propaganda_type'].unique()
prop_type_list


array(['Name_Calling,Labeling', 'Loaded_Language',
       'Appeal_to_fear-prejudice', nan, 'Slogans', 'Repetition',
       'Whataboutism', 'Causal_Oversimplification',
       'Exaggeration,Minimisation', 'Doubt',
       'Thought-terminating_Cliches', 'Flag-Waving',
       'Black-and-White_Fallacy', 'Reductio_ad_hitlerum',
       'Appeal_to_Authority', 'Red_Herring',
       'Obfuscation,Intentional_Vagueness,Confusion', 'Straw_Men',
       'Bandwagon'], dtype=object)

In [29]:
#filter for each kind of propaganda
propaganda[propaganda['propaganda_type']=='Name_Calling,Labeling'][['article_id','sent_#','text']]


,article_id,sent_#,text
1,701225819,2,"David Duke, the white supremacist icon and former Grand Wizard of the Ku Klux Klan, has been denounced by many as a hatemonger, and rightfully so."
8,701225819,9,"Three of the videos were put out by Duke’s official website, DavidDuke.com, one referring to CNN news anchor Wolf Blitzer as a “Jewish Zionist Agent.” The fourth video was a news report featuring a clip of Duke being interviewed at a white nationalist rally held in Charlottesville, Virginia this..."
21,701225819,22,"No, that’s Satan."
22,701225819,23,"You should learn to call them by their real name, ‘Satan.’ You’re coming face to face with Satan, the Arch Deceiver, the enemy of God, and the enemy of the righteous.”"
31,701225819,32,"Soup to nuts, top to bottom, the 9/11/01 terrorist attacks on New York City and Washington D.C. were an ‘Israeli’-Jewish job.” The website which published the report and which Zakkout linked to is Mouqawamah Music, a site that openly calls for “Death to Israel” and labels the Jewish religion “wi..."
32,701225819,33,"In February 2016, Zakkout circulated on social media a report claiming that “the Holocaust was faked.” It begins: “The alleged ‘Holocaust’ of ‘6 million Jews’ at the hands of Adolf Hitler and National Socialist Germany during WWII is the biggest lie ever foisted upon humanity.” It was produced b..."
34,701225819,35,"Above the picture of Eizenkot, Zakkout wrote in Arabic, “You’re a Jew, the grandson of a monkey and a pig.” He signed it “Sofian.” This is one of many times Zakkout has referred to Jews as “monkeys and pigs.”"
297,782086447,217,"He followed the advice of someone he knew well to be a pervert, thus multiplying exponentially with his supreme authority the evil done by McCarrick."
426,729303442,18,"Democratize SSMU deleted the message, which targeted Lew and two other candidates, after the meeting and apologized for publishing material that was “insensitive to anti-Semitic tropes of Jewish people as corrupt and politically powerful.”"
432,729303442,24,"Michael Mostyn, CEO of B’nai Brith Canada, called the report a whitewash."


In [18]:
prop_type

,0,1
0,Appeal_to_Authority,...
1,Appeal_to_fear-prejudice,...
2,Bandwagon,text...
3,Black-and-White_Fallacy,...
4,Causal_Oversimplification,...
5,Doubt,...
6,"Exaggeration,Minimisation",...
7,Flag-Waving,...
8,Loaded_Language,...
9,"Name_Calling,Labeling",...
